# Predicting Overdraft Risk from Personal Transactions

**Problem:** Predict which accounts are likely to go into overdraft next month based on transactions and budget data.  

**Goal:** Demonstrate end-to-end data science workflow: data cleaning, feature engineering, modeling, evaluation, and insights.  

**Use Cases:** This model helps users manage their finances and avoid overdraft fees. Future iterations could provide personalized saving recommendations based on spending categories.


Import relevant libraries

In [11]:
import pandas as pd
from pandas import DataFrame, Series
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from numpy import ndarray
from typing import Dict

Create DataFrames

In [12]:
budget: DataFrame = pd.read_csv(r"C:\Users\anita\Desktop\DataScience\data\Budget.csv")

budget.head()

,Category,Budget
0,Alcohol & Bars,50
1,Auto Insurance,75
2,Coffee Shops,15
3,Electronics & Software,0
4,Entertainment,25


In [13]:
transactions: DataFrame = pd.read_csv(r"C:\Users\anita\Desktop\DataScience\data\personal_transactions.csv")

transactions.head()

,Date,Description,Amount,Transaction Type,Category,Account Name
0,01/01/2018,Amazon,11.11,debit,Shopping,Platinum Card
1,01/02/2018,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,01/02/2018,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,01/03/2018,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,01/04/2018,Netflix,11.76,debit,Movies & DVDs,Platinum Card


Next I cleaned the data with the following steps:

1.Strip extra spaces from column names

2.Fix date formats

3.Remove duplicates or irrelevant columns

In [14]:
transactions.columns = transactions.columns.str.strip()
budget.columns = budget.columns.str.strip()

transactions['Date'] = pd.to_datetime(transactions['Date'], dayfirst=False)

Next I normalized the category names

In [15]:
category_mapping: Dict[str, str] = {
    "Restaurants": "Food & Dining",
    "Fast Food": "Food & Dining",
    "Coffee Shops": "Food & Dining",
    "Movies & DVDs": "Entertainment",
    "Music": "Entertainment",
    "Television": "Entertainment",
    "Shopping": "Shopping",
    "Home Improvement": "Home Improvement",
    "Electronics & Software": "Electronics",
    "Internet": "Utilities",
    "Mobile Phone": "Utilities",
    "Gas & Fuel": "Transport",
    "Auto Insurance": "Transport",
    "Alcohol & Bars": "Alcohol & Bars",
    "Haircut": "Personal Care",
    "Mortgage & Rent": "Rent",
    "Credit Card Payment": "Credit Card Payment",
    "Utilities": "Utilities",
    "Paycheck": "Income",
}

transactions["Category_clean"] = (
    transactions["Category"]
    .map(category_mapping)
    .fillna(transactions["Category"])
)

budget["Category_clean"] = (
    budget["Category"]
    .map(category_mapping)
    .fillna(budget["Category"])
)

Then merged the data sets

In [16]:
merged: DataFrame = transactions.merge(
    budget,
    on="Category_clean",
    how="left"
)

Then:

Calculated monthly spend per category and extracted the month

Merged with budget csv to compare and dropping duplicate rows

Counted cattegories that went over the budget

Created the target variable and set the risk threshold

In [17]:

merged['Month'] = merged['Date'].dt.to_period('M')

monthly_cat_spend: DataFrame = merged.groupby(['Account Name', 'Month', 'Category_clean'])['Amount'].sum().reset_index()

monthly_cat_spend = monthly_cat_spend.merge(
    budget[['Category_clean', 'Budget']].drop_duplicates(), 
    on='Category_clean', 
    how='left'
)

monthly_cat_spend['is_over'] = monthly_cat_spend['Amount'] > monthly_cat_spend['Budget']

over_budget_counts: DataFrame = (
    monthly_cat_spend.groupby(['Account Name', 'Month'])['is_over']
    .sum()
    .reset_index(name='over_budget_count')
)

# --- Main Monthly Aggregation ---
monthly: DataFrame = merged.groupby(['Account Name', 'Month']).agg(
    total_spent=pd.NamedAgg(column='Amount', aggfunc='sum'),
    transactions_count=pd.NamedAgg(column='Amount', aggfunc='count')
).reset_index()

# Add the corrected over_budget count
monthly = monthly.merge(over_budget_counts, on=['Account Name', 'Month'], how='left')
monthly['over_budget_count'] = monthly['over_budget_count'].fillna(0)

monthly = monthly.sort_values(['Account Name', 'Month'])

monthly['next_month_overdraft'] = monthly.groupby('Account Name')['total_spent'].shift(-1) > 2000
monthly['next_month_overdraft'] = monthly['next_month_overdraft'].fillna(False)

One-hot encoded 'Account Names' column

In [18]:
monthly = pd.get_dummies(monthly, columns=['Account Name'])


Train/Test + Model:

Sorted by month

Defined the split size

Time-Series split

Then trained the model

Then predicted with test data

In [19]:

monthly = monthly.sort_values('Month')

X: DataFrame = monthly.drop(columns=['Month', 'next_month_overdraft'])
y: Series = monthly['next_month_overdraft']

train_size: int = int(len(monthly) * 0.8)

X_train: DataFrame
X_test: DataFrame
y_train: Series
y_test: Series

X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]

model: RandomForestClassifier = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred: ndarray = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       1.00      0.89      0.94         9
        True       0.80      1.00      0.89         4

    accuracy                           0.92        13
   macro avg       0.90      0.94      0.92        13
weighted avg       0.94      0.92      0.93        13



Analysis:

The classification report shows the model is performing strongly overall, but it handles each class slightly differently. It gets 92% of all predictions correct, with more data in the low-spend class than the high-spend class.

For low-spend months, the model is extremely accurate: it never falsely labels a high-spend month as low (precision 1.00), and it correctly identifies most of the actual low-spend months (recall 0.89). It only misclassified one.

For high-spend months, the model catches all of them (recall 1.00), which is ideal for an overdraft-risk scenario. The trade-off is that it occasionally flags a low-spend month as high-spend (precision 0.80), but this is acceptable because missing a genuinely risky month would be worse than sending an occasional unnecessary warning.

Overall, the model prioritises correctly detecting every high-risk month, and its precision/recall balance makes it well-suited for alerting users before they overspend.

Limitations:
Very small dataset so the metrics aren’t reliable or generalisable.

Simple features and one-hot encoding increases the risk of overfitting.

The “high spending = risk” target is based on a basic threshold and doesn’t reflect real banking behaviour.

Not production-ready: no error handling, no real-time pipeline, and limited feature engineering.

Improvements:
Use a minimum running balance for the month instead of a fixed spending threshold. 

Stronger Time-Series Features
Add features that capture short-term behaviour shifts:
EWMA of spending (30–90 days)
Standard deviation of daily spend to measure volatility

Add Explainability (XAI)
Use SHAP or LIME so predictions are interpretable and actionable, improving user trust and transparency.